# Class BoolRepres 

Instances of this class represents a  binary encopde of a natural number with length at most ```n_bits```.

In [2]:
from math import log2
class BoolRepres() :
    
    #==============================================================
    #----------------------------- NB -----------------------------
    # Pensata per RAPPRESENTARE COMODAMENTE relazioni BINARIE 
    # only INTEGER value
    
    def __init__(self, varName, nBits=1):
        
        self.nBits = nBits
        self.varName = varName
        self.bitsName =  [ (str(varName)+'_')+str(nBits-1-i) for i in range(nBits)]
        self.bits = self.bitsName
        
        # ckeck if varName can be int
        try:
            new_br = self.set_to(int(varName))
            self.bitsName, self.bits = new_br.bitsName, new_br.bits         
        except :
            pass
        
    #------------------------------------------
    def __str__(self):
        return "b"+str(self.bits)
    def __repr__(self):
        return self.__str__()
    def __getitem__(self, key):
        return self.bits[key]
    def __len__(self):
        return self.nBits
       
    #------------------------------------------
    def get_bits(self):
        return self.bits
    
    def get_bitsName(self):
        return self.bitsName
    
    def reset_bits(self):
        self.bits = self.bitsName
        
    def to_dict(self):
        temp = {}
        for k,v in zip(self.bitsName, self.bits):
            temp[k]=v
        return temp
    
    def set_to(self, val:int, reduce=False): #val:int
        
        # evito problei di cast (es numpy.Int != int ecc)
        val = int(val) 
        
        # num Bits between original dimension and new_bits + evito log(0)
        new_nBits = int(log2(max(1,val)))+1
        if reduce==False:
            new_nBits = max( new_nBits, self.nBits) 
        new_Bits = ['true' if b=='1' else 'false' for b in bin(val)[2:].zfill( new_nBits )] 
        
        # create new BoolRepres2
        new_br = BoolRepres(str(self.varName), new_nBits)
        new_br.bits = new_Bits
        return new_br
               
    #==============================================================
    #------------------- Bit check and Conversion -----------------
    def __checkType(self, other):
        if isinstance(other, type(self)): return other   
        
        # Try a cast to int
        try:                       # NB: log2(0) = error
            return BoolRepres(int(other), int(log2( max(1,other) ))+1) 
        except:
            print(f'TypeError: Variable is type:{type(other)}. Only myBoolRepres or int type')
            return None    
    
    #--------------------------------------------------
    def __checkLen(self, other):
        if self.nBits == other.nBits: 
            return self.bits, other.bits
        if self.nBits < other.nBits:
            return ['false']*(other.nBits - self.nBits)+self.bits, other.bits  
        else:
            return self.bits, ['false']*(self.nBits - other.nBits)+other.bits 
               
    #==============================================================
    #-------------------- Override operators ----------------------
    def __lessEqual(self, other): # oper: self <= other
        temp, formula = '', [] 
        for self_k, other_k in zip(*self.__checkLen(other)):
            f= f"(not {self_k} or {other_k}" + temp + ")" #NB: temp='' nella prima iterazione. 
            temp += f" or (not {self_k} and {other_k})"    # Dalla seconda in poi, aggiunge info sui numeri precedenti all'k-esimo bit
            formula.append(f)
        return  "(" + " and ".join(formula) +")"
        
    #--------------------------------------------------
    def __lessThan(self, other): # oper: self < other  
        temp, formula = '', [] 
        for self_k, other_k in zip(*self.__checkLen(other)):
            f= f"(not {self_k} or {other_k}" + temp + ")" #NB: temp='' nella prima iterazione. 
            temp += f" or (not {self_k} and {other_k})"    # Dalla seconda in poi, aggiunge info sui numeri precedenti all'k-esimo bit
            formula.append(f)
            
        # Correggo ultima iterazione: Rifiuto il caso self==other
        formula[-1]='(false'+temp+')'
        return  "(" + " and ".join(formula) +")"
       
    #--------------------------------------------------
    def __Equal(self, other): # oper: ==
        formula = []
        if isinstance(other, type(self)): 
            for self_k, other_k in zip(*self.__checkLen(other)):
                f= f"((not {self_k} and not {other_k}) or ({self_k} and {other_k}))"
                formula.append(f)
            return "(" + " and ".join(formula) + ")" 

        # Se qui, sicuramente caso int -> unico altro caso considerato
        for self_k, other_k in zip(*self.__checkLen(self.__checkType(other))):
            f= f"{self_k}" if other_k=='true' else f"not {self_k}"
            formula.append(f)              
        return "(" + " and ".join(formula) + ")" 
    
    #--------------------------------------------------
    def __eq__(self, other):
        return self.__Equal(self.__checkType(other))
    #--------------------------------------------------
    def __le__(self, other):
        return self.__lessEqual(self.__checkType(other))
    #--------------------------------------------------
    def __lt__(self, other):
        return self.__lessThan(self.__checkType(other))
    #--------------------------------------------------
    def __ge__(self, other): # self >= other <--> other <= self, 
        return self.__checkType(other) <= self   
    #--------------------------------------------------
    def __gt__(self, other): # self > other <--> other < self
        return  self.__checkType(other) < self 
    #--------------------------------------------------
    def __ne__(self, other): # self != other <--> not(self == other)
        return f'(not{self == self.__checkType(other)})' 
        
    
    #--------------------------------------------------
    #-------------------- TEMP ------------------------
    def le_cnf(self, other):
        temp, supp_cnf, formula ='', [], [] 
        other = self.__checkType(other)
       
        for i,(self_k, other_k) in enumerate(zip(*self.__checkLen(other))):
            formula.append( f"(not {self_k} or {other_k}" + temp + ")")  # NB: temp='' nella prima iterazione. 
            
            # per cnf => name_cnf_k <==> not {self_k} and {other_k}
            name_cnf = f"{self.varName}_{str(other.varName)}_{i}"
            temp += f" or {name_cnf}"    # Dalla seconda in poi, aggiunge info sui numeri precedenti all'k-esimo bit
            supp_cnf.append(f"(not {name_cnf} or not {self_k}) and (not {name_cnf} or {other_k})") # implicazione: --> 
            supp_cnf.append(f"({self_k} or {other_k} or {name_cnf})")                              # implicazione <--
            
        return  "(" + " and ".join(formula+supp_cnf) +")"
        
        